# 2017-05-22 : manipulations and processing of the video
---
I still have the connection problems I have every morning. Need to restart my computer a couple time until it works again. Always around 9.

Since I have huge problems with the *openCV* python module since last week, I may take a look at *MoviePy*, another powerful video editing module (all informations at : http://zulko.github.io/moviepy/). I'll only use it to display the video as a control, so it doesn't matter which one I'm using.  

First, I need to install MoviePy (more informations : http://zulko.github.io/moviepy/install.html) : 

    pip3 install moviepy --user
    
That module seems good to display the video and later I could have used it to help me in the video frame to frame analysis, but it seems to not be good for that kind of tasks (see : http://zulko.github.io/moviepy/getting_started/quick_presentation.html#do-i-need-moviepy).  
Plus it wasn't regularly updated for years (see : https://www.reddit.com/r/moviepy/comments/5snfks/is_moviepy_so_outdated_that_it_no_longer_works/ and https://www.reddit.com/r/moviepy/comments/5q65w9/still_being_developed/) until some months ago (see : https://github.com/Zulko/moviepy/graphs/contributors).  

An alternative would be *ImageIO* (all informations at : http://imageio.github.io/).  

With MoviePy, displaying a video should be as simple as that script, but it need some dependancies : 

    from moviepy import *
    from moviepy.editor import *

    clip = VideoFileClip('video.avi')
    clip.show()
    =================================
    ImportError: clip.show requires Pygame installed
    
So : 

    pip3 install Pygame --user
    
The script now display the video but I have two problems : 
- The video I used is static, so I can't see if its showing the whole video or just a frame
- The displaying seems to freeze and I cant close it, even by interrupting the kernel, I need to restart it. I must add some closing of killing lines

The freezing display was easily solved by remplacing : 

    clip.show()
    
By : 

    clip.ipython_display
    
And it even allow me to display videos into ipython notebook !  
I note that the video is supposed to be 5 seconds long but the display shows it for 6s, but the last second looks like a freeze of the last frame. That last second could be the product of any of the program steps.  

To compare frames between them (aka do a frame-by-frame traitment), I need to find a way to sort frames individually.  
*MoviePy* have a native command to do this, *iter_frames()* (More informations : https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html#moviepy.video.io.VideoFileClip.VideoFileClip.iter_frames and https://www.reddit.com/r/moviepy/comments/623pcr/is_this_giving_me_the_data_for_a_single_frame/) :

    for frame in video.iter_frames():
        #Traitment
        Print(frame)
    =================================
    [[[62 74 78]
      [62 74 78]
      [62 74 78]
      ..., 
      [ 6  8  0]
      [ 3  5  0]
      [ 0  0  0]]

     [[62 74 78]
      [62 74 78]
      [62 74 78]
      ..., 
      [ 6  8  0]
      [ 3  5  0]
      [ 0  0  0]]

     [[62 74 78]
      [62 74 78]
      [62 74 78]
      ..., 
      [ 6  8  0]
      [ 3  5  0]
      [ 0  0  0]]

     ..., 
     [[67 66 55]
      [67 66 55]
      [67 66 55]
      ..., 
      [16 11  0]
      [15 10  0]
      [ 0  0  0]]

     [[67 66 55]
      [67 66 55]
      [67 66 55]
      ..., 
      [16 11  0]
      [15 10  0]
      [ 0  0  0]]

     [[67 66 55]
      [67 66 55]
      [67 66 55]
      ..., 
      [16 11  0]
      [15 10  0]
      [ 0  0  0]]]
      
Now I can try to expose these frames to the same traitment as the one that eventRetina uses.  
Some informations about *numpy.hstack()* (that could be replaced by *numpy.concatenate(tup,axis=1)* or *numpy.stack()*) : https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.hstack.html  

At that point, the script seems to correctly iterate to each frame of the video, but correctly edit the values only of the first one. It is a problem and I don't understand what's causing it. Plus, the writing file line is not working as expected : for now its copying the video.

# 2017-05-23
---
Added comments to the *_capture_videos_server* scripts to help potential futur users.  
Since few days, I'vg got some computer problems : in addition to the need to reboot it around 9 everyday because of internet issues, it's now slower to boot and much slower to push files through git (more than 10 minutes).  

Some informations about *numpy.ceil()* : https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.ceil.html  
Some informations about *numpy.sum()* : https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.sum.html  
Some informations about *numpy.argsort()* : https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html  

When the following line is deleted (or put as comment) :

    frame = frame.sum(axis=-1)
    =====================================================================================
    ValueError: operands could not be broadcast together with shapes (480,640,3) (480,640) 
    
So it seem necessary to have the same dimensions for *frame* and *frame_old*. But it modify the data :

    Before : 
    frame :  [[[ 62.  74.  78.]
      [ 62.  74.  78.]
      [ 62.  74.  78.]
      (...)
    
    After :
    frame :  [[ 214.  214.  214. ...,   14.    8.    0.]
     [ 214.  214.  214. ...,   14.    8.    0.]
     [ 214.  214.  214. ...,   14.    8.    0.]
     (...)
     
I think that step can be skipped by directly defining the *frame_old* data the way the *frame* data are :
 
     Before :
     frame_old = np.zeros((h, w))
     
     After:
     frame_old = np.zeros((h, w, 3))
     
Some informations about numpy.ravel() : https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.ravel.html  
I need to edit the part of the script that convert the video file to .avi, because its an outdated format and now it's vastly replaced with .mp4 (much more compact). Plus the avi format was developed for Windows so using it may cause conflicts on other operating systems.  
I note that the video files don't show if another computer open my ipynb, it needs to run the script first. 

About the lines : 

    frame_old = np.zeros((h, w, 3))
    (...)
    #frame = frame.sum(axis=-1)
    
    OR
    
    frame_old = np.zeros((h, w)
    (...)
    frame = frame.sum(axis=-1)
    
The two versions allows to chose between a colored video and a grey-scaled video.  
The fact that have difficulties modifying the video may come from a specifity of *frames*

    print(len(frames))
    ==================================================
    TypeError: object of type 'generator' has no len()
    
More informations about generators : https://wiki.python.org/moin/Generators and https://stackoverflow.com/questions/1756096/understanding-generators-in-python  

Actually it seems that I'm able to modify the frames values up to that line :

    frame = np.zeros((h, w, 3), dtype = np.uint8)
    =============================================
    [[[0 0 0]
      [0 0 0]
      [0 0 0]
      (...)
      
So the following lines aren't able to modify the frames values further than the first frame:

    frame[:, :, 0][np.unravel_index(data[:n_datapoints], (h, w))] = 255
    frame[:, :, -1][np.unravel_index(data[-n_datapoints:], (h, w))] = 255
    
Actually, it seems that few frames contain the right values to get modified, as :

    print(frame)
    ===============
    frame  77
    [[255   0   0]
      [255   0   0]
      [255   0   0]
    (...)

So the two lines previously cited may work correctly.

The fl_image() function (more informations : https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html#moviepy.video.io.VideoFileClip.VideoFileClip.fl_image) should allow to remplace frames, but when calling it :

    video.fl_image()
    =========================================================================
    TypeError: fl_image() missing 1 required positional argument: 'image_func'

Then : 

    video.fl_image(frame)
    =================================================
    TypeError: 'numpy.ndarray' object is not callable

I may take a look at matplotlib image manipulations : https://matplotlib.org/users/image_tutorial.html  
I had to install matplotlib :

    pip3 install matplotlib --user
    


# Outline
---
- [X] Run a display of the video file
    - [X] Write an openCV script to run the video file
    - [X] Convert the h264 format file to one readable by openCV
    - [CANCELED] Make openCV read the video file
    - [X] Make MoviePy read the video file
    - [X] Capture and convert a non-static video
    - [X] [OPT] Find a way to debug the display freeze
- [X] Find a way to separate frames of the video
- [ ] Introduce comparaisons between frames to only code moving pixels
    - [X] Solve an issue : script correctly modifying only the first frame
    - [ ] Solve an issue : "write file" command don't contain modifications
- [X] Convert the video file to mp4 instead of avi
- [X] Capture a video with a stable background and moving object
- [ ] Implement the scripts into OpenRetina
- [ ] Make the Pi do the GC layer work 
- [ ] [OPT] Add a way to delete the video file on the Pi after sending and processing are done.
- [ ] [OPT] Try to find the optimal chunk size for a faster file transfer.
- [ ] [OPT] Rewrite the scripts to introduce classes
- [ ] [OPT] Dive into the video processing steps to find out when the sixth second (if the video is 5s long) is added

Going further :
- [ ] Study the program bottlenecks (Where does the program lose computing time?)
    - [ ] Add flags to monitors each script
- [ ] Switch from synchronous (phrs grabbing and sending frames when receiving other layers requests) to asynchronous (phrs grabbing continously and sending frames when requested) processes